# QC protocol for Private Weather Stations

This notebook presents how to use the Python package `pypwsqc`, a quality assurance protocol developed for automated private weather stations (PWS).
The protocol consists of three filters from de Vos et al (2019) the Faulty Zero filter, the High Influx filter and the Station Outlier filter as well as the Indicator Correlation Filter (IC) from Bardossy et al. (2021) 



Publications: 
* de Vos, L. W., Leijnse, H., Overeem, A., & Uijlenhoet, R. (2019). Quality control for crowdsourced personal weather stations to enable operational rainfall monitoring. Geophysical Research Letters, 46(15), 8820-8829 with original R code at available at https://github.com/LottedeVos/PWSQC/.
* Bárdossy, A., Seidel, J., and El Hachem, A.: The use of personal weather station observations to improve precipitation estimation and interpolation, Hydrol. Earth Syst. Sci., 25, 583–601, https://doi.org/10.5194/hess-25-583-2021, 2021. 



In [ ]:
import sys

sys.path.append('poligrain/src')
sys.path.append('pypwsqc/src')

In [ ]:
import matplotlib.pyplot as plt
import poligrain as plg
import xarray as xr
from tqdm import tqdm
import numpy as np

import pypwsqc

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

## Download example data

In this example, we use an open PWS dataset from Amsterdam, called the "AMS PWS" dataset. By running the cell below, a NetCDF-file will be downloaded to your current repository (if your machine is connected to the internet).

In [ ]:
import os
if not os.path.exists("example_data/"):
    os.makedirs("example_data/")

In [ ]:
!curl -L https://github.com/OpenSenseAction/OS_data_format_conventions/raw/main/notebooks/data/OpenSense_PWS_example_format_data.nc > example_data/OpenSense_PWS_example_format_data.nc


## Data preparations

This package handles rainfall data as `xarray`  Datasets. The data set must have `time` and `id` dimensions, `latitude` and `longitude` as coordinates, and `rainfall` as data variable.

An example of how to convert .csv data to a `xarray` dataset is found [here](https://github.com/OpenSenseAction/OS_data_format_conventions/blob/main/notebooks/PWS_example_dataset.ipynb).

In [ ]:
ds_pws = xr.open_dataset("example_data/OpenSense_PWS_example_format_data.nc").load()
ds_pws

### Reproject coordinates
First we reproject the coordinates to a local metric coordinate reference system to allow for distance calculations. In the Amsterdam example we use EPSG:25832. **Remember to use a local metric reference system for your use case!** We use the function `spatial.project_point_coordinates` in the `poligrain`package. 

In [ ]:
ds_pws.coords["x"], ds_pws.coords["y"] = plg.spatial.project_point_coordinates(
    x=ds_pws.longitude, y=ds_pws.latitude, target_projection="EPSG:25832"
)

### Create distance matrix

Then, we calculate the distances between all stations in our data set. If your data set has a large number of stations this can take some time.

In [ ]:
distance_matrix = plg.spatial.calc_point_to_point_distances(ds_pws, ds_pws)

In [ ]:
plt.pcolor(distance_matrix.values)
plt.colorbar(label='distance [m]')
plt.xlabel('pws_id')
plt.ylabel('pws_id');

### Calculate data variables 
Next, we will calculate the data variables `nbrs_not_nan` and `reference` that are needed to perform the quality control.

`nbrs_not_nan`:
Number of neighbours within a specificed range `max_distance` around the station that are reporting rainfall for each time step. The selected range depends on the use case and area of interest. In this example we use 10'000 meters. 

 `reference`:
Median rainfall of all stations within range `max_distance` from each station.

In [ ]:
max_distance = 10e3

In [ ]:
nbrs_not_nan = []
reference = []

for pws_id in tqdm(ds_pws.id.data):
    neighbor_ids = distance_matrix.id.data[
        (distance_matrix.sel(id=pws_id) < max_distance)
        & (distance_matrix.sel(id=pws_id) > 0)
    ]

    N = ds_pws.rainfall.sel(id=neighbor_ids).isnull().sum(dim="id")
    nbrs_not_nan.append(N)

    median = ds_pws.sel(id=neighbor_ids).rainfall.median(dim="id")
    reference.append(median)

ds_pws["nbrs_not_nan"] = xr.concat(nbrs_not_nan, dim="id")
ds_pws["reference"] = xr.concat(reference, dim="id")

### Load reference data set
As referecence data for the indicator correlation filter (so called primary stations, c.f. Bárdossy et al. (2021)), 20 time series from pixels from the gauge-adjusteed KNMI radar product over the Amsterdam Metropolitan area were chosen randomly.

The following cell loads this data set and adds cartesian coordinates as shown above

In [ ]:
ds_ref = xr.open_dataset("pypwsqc/docs/notebooks/data/RadarRef_AMS.nc")
ds_ref.load()

ds_ref.coords["x"], ds_ref.coords["y"] = plg.spatial.project_point_coordinates(
    ds_ref.lon,
    ds_ref.lat,
    target_projection="EPSG:25832",
)

## Exercise 1
1.1 Plot one or more time series of PWS and reference data  
1.2 Calculate and plot the distance between PWS and reference data  
1.3 Plot an approximate and a widely seperated pair of PWS-refernce data

In [ ]:
# 1.1 - your solution here



In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/3_1_1_solution.py

In [ ]:
# 1.2 - your solution here



In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/3_1_2_solution.py

In [ ]:
# 1.3 - your solution here



In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/3_1_3_solution.py


## Faulty Zero Filter (FZ)
Conditions for raising Faulty Zeros flag:

* Median rainfall of neighbouring stations within range `max_distance` is larger than zero for at least `nint` time intervals while the station itself reports zero rainfall.
* The FZ flag remains 1 until the station reports nonzero rainfall.
* Filter cannot be applied if less than `nstat` neighbours are reporting data (FZ flag is set to -1)

For settings for parameter `nint` and `nstat`, see table 1 in [de Vos et al. (2021)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019GL083731)

In [ ]:
%%time 
# takes 2-3 minutes
# compute filter
fz_flag = pypwsqc.flagging.fz_filter(
    pws_data=ds_pws.rainfall,
    nbrs_not_nan=ds_pws.nbrs_not_nan,
    reference=ds_pws.reference,
    nint=3,
    n_stat=5,
)

#add flag to ds_pws
ds_pws["fz_flag"] = fz_flag

## High Influx Filter (HI)
Conditions for raising High Influx flag:

* If median below threshold `ϕA`, then high influx if rainfall above threshold `ϕB`
* If median above `ϕA`, then high influx if rainfall exceeds median times `ϕB`/`ϕA`
* Filter cannot be applied if less than `nstat` neighbours are reporting data (HI flag is set to -1)

For settings for parameter `ϕA`, `ϕB` and `nstat`, see table 1 in [de Vos et al. (2021)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019GL083731)

In [ ]:
%%time
# compute filter
hi_flag = pypwsqc.flagging.hi_filter(
    pws_data=ds_pws.rainfall,
    nbrs_not_nan=ds_pws.nbrs_not_nan,
    reference=ds_pws.reference,
    hi_thres_a=0.4,
    hi_thres_b=10,
    n_stat=5,
)

#add flag to ds_pws
ds_pws["hi_flag"] = hi_flag

## Exercise 2
2.1 Plot the occurance of the flags over time and PWS ids.  
2.2 Plot a map of the pws and reference stations, and their rainfall sum with and without the flagged data. What is the difference?

In [ ]:
# 2.1 - your solution here



In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/3_2_1_solution.py

In [ ]:
# 2.2 - your solution here



In [ ]:
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/3_2_2_solution.py

## Indicator Correlation Filter (IC)
The PWS data needs to be in hourly values as the indocator correlation filter by Bárdossy et al. (2021) does not work with 5 minute data

For the aggreation, the new value for an hour is considered as valid if at least 10 out 12 5-min values within one hour have valid data. This can be set by the min count parameter.

In [ ]:
ds_pws_hourly = ds_pws.resample(time="1h").sum(min_count=10)


### Indicator correlation vs distance
First, we calculate the indicator correlations over distance for the reference data set. This is assumed to be the correct spatial pattern of precipitation which is used for checking the PWS later on.

In [ ]:
import pypwsqc.indicator_correlation as ic

In [ ]:
# Distance and indicator correlations of reference data
dist_mtx_ref, indcorr_mtx_ref = ic.indicator_distance_matrix(
    ds_ref.rainfall,
    ds_ref.rainfall,
    max_distance=30e3,
    prob=0.99,
    min_valid_overlap=2 * 24 * 30,
)

In [ ]:
# plot the indicator time series of all reference stations
(ds_ref.rainfall>ds_ref.rainfall.quantile(0.99)).plot(figsize=(15,5));

In [ ]:
plt.scatter(dist_mtx_ref, indcorr_mtx_ref, color="red", s=10, label="Ref")
plt.ylim(0, 1)
plt.xlim(0, 30e3)
plt.ylabel("Indicator Correlation [-]")
plt.xlabel("Distance [m]")
plt.title("Indicator Correlation vs. Distance for Reference Data")
plt.legend();

In [ ]:
# Distance and indicator correlations of PWS
dist_mtx_pws, indcorr_mtx_pws = ic.indicator_distance_matrix(
    ds_pws_hourly.rainfall,
    ds_pws_hourly.rainfall,
    prob=0.99,
    max_distance=30e3,
    min_valid_overlap=2 * 24 * 30,
)

In [ ]:
plt.scatter(dist_mtx_pws, indcorr_mtx_pws, color="b", alpha=0.2, s=10, label="PWS")
plt.scatter(dist_mtx_ref, indcorr_mtx_ref, color="red", s=10, label="Ref")
plt.ylim(0, 1)
plt.xlim(0, 30e3)
plt.ylabel("Indicator Correlation [-]")
plt.xlabel("Distance [m]")
plt.title("Indicator Correlation over Distance for PWS and Reference Data")
plt.legend()
;

We can see that the PWS data is very "noisy", i.e. the indicator correlation of nearby PWS stations is very low which we would not expect from the reference. Such PWS are likely to have data quality issues ans will be removed by the Indicator Correlation Filter.

Finally the distance and indicator correlations matrices between PWS and reference data are calculated.

In [ ]:
dist_mtx_ref_pws, indcorr_mtx_ref_pws = ic.indicator_distance_matrix(
    da_a=ds_ref.rainfall,
    da_b=ds_pws_hourly.rainfall,
    prob=0.99,
    min_valid_overlap=2 * 24 * 30,
)

### Apply filter

In [ ]:
indcorr_results = ic.ic_filter(
    indicator_correlation_matrix_ref=indcorr_mtx_ref,
    distance_correlation_matrix_ref=dist_mtx_ref,
    indicator_correlation_matrix=indcorr_mtx_ref_pws,
    distance_matrix=dist_mtx_ref_pws,
    max_distance=20000,
    bin_size=1000,
    quantile_bin_ref=0.1,
    quantile_bin_pws=0.5,
    threshold=0.05,
);

In [ ]:
indcorr_results

The results are returned as `xarray.Dataset` with four variables:

`indcorr`: Indicator correlation matrix between `Ref` and `PWS`

`dist`: Distance matrix between `Ref` and `PWS`

`indcorr_good`: Bool Array indicating whether a PWS was accepted ('True') or rejected ('False') by the filter

`indcorr_score`: A metric which indicates how well a PWS fit's into the correlation structure of the Reference

In [ ]:
print(
    str(indcorr_results.indcorr_good.data.sum())
    + " of "
    + str(len(indcorr_results.indcorr_good))
    + " PWS were accepted"
)

## Exercise 3
3.1 Plot time series of following pws and colorize their points in the ic vs distane plot: 'ams74', 'ams134', 'ams113', 'ams36'.      
3.2 Check if these pws where flagged by the FZ or HI filter.  
3.3 Find the PWS with the most HI flags and show add it to the list of ids.  

In [ ]:
# 3. your solution here



In [ ]:
%matplotlib widget

In [ ]:
# %load solutions/3_3_1-3_solution.py
if input("Enter 'Solution' to display solutions: ")=='Solution':
    %load solutions/3_3_1-3_solution.py